https://www.kaggle.com/code/pavansanagapati/spotify-music-api-data-extraction-part1/notebook

In [62]:
#To access Spotify authorised data we need to create our credentials like clientid and client_secret as well as spotify object to access the API

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
# username = 'ray1d5u3xeb2e8c1nkpnwsnig'
client_id = 'bbf8d106196049c7831e658b73eb7a72'
client_secret = 'a249efe0cd3c411fa6182dfa2f4ce048'
# redirect_url = 'http://localhost:7777/callback'
# scope = 'user-read-recently-played'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
name = "Juice WRLD"
result = sp.search(name) 
result['tracks']['items'][1]['artists']

In [ ]:
type(result)

### Extract Spotify Albums

We will store Spotify URIs and album names in separate lists for reference that we can use later on during data analysis

In [ ]:
#Extract Artist's uri
artists_uris = result['tracks']['items'][0]['artists'][0]['uri']

In [ ]:
artists_uris

In [ ]:
type(artists_uris)

In [ ]:
#Pull all of the artist's albums
artist_albums = sp.artist_albums(artists_uris, album_type='album')

In [ ]:
artist_albums

In [ ]:
#Store artist's albums' names' and uris in separate lists
artist_album_names = []
artist_album_uris = []
for i in range(len(artist_albums['items'])):
    artist_album_names.append(artist_albums['items'][i]['name'])
    artist_album_uris.append(artist_albums['items'][i]['uri'])
    
artist_album_names
#Keep names and uris in same order to keep track of duplicate albums

In [ ]:
artist_album_uris

In [ ]:
artist_album_uris[0]

In [ ]:
sp.album_tracks(artist_album_uris[0])

### Extract all the songs from every album

Next would be to loop through each album to extract key album track data.

In [ ]:
def album_songs(uri):
    album = uri 
    spotify_albums[album] = {}
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] 
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    #pull data on album tracks
    tracks = sp.album_tracks(album) 
    for n in range(len(tracks['items'])): 
        spotify_albums[album]['album'].append(artist_album_names[album_count]) 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

Now we can apply the function to each album URI in the list to pull the track data and create an empty dictionary called to store the spotify album data.

In [ ]:
spotify_albums = {}
album_count = 0
for i in artist_album_uris: #each album
    album_songs(i)
    print(str(artist_album_names[album_count]) + " album songs has been added to spotify_albums dictionary")
    album_count+=1 #Updates album count once all tracks have been added

### Extract audio features for each song

To store the audio features of each album track and append the data into lists representing all the music tracks for that album we add additional key-values.

In [ ]:
def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        
        #Append to relevant key-value
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

Now let loop through albums extracting the audio features.For this we will add a random delay every few albums to avoid sending too many requests at Spotify's API.

In [ ]:
import time
import numpy as np
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    audio_features(i)
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " playlists completed")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

Now its time to add the above spotify music data to a new dataframe but before we do so we need to first organise our data into a dictionary which can be converted into a dataframe.

In [ ]:
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])
        
len(dic_df['album'])

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(dic_df)
df[0:5]

Spotify has a duplicate issue which can be addressed by removing all but the most popular songs.

In [ ]:
print(len(df))
final_df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
print(len(final_df))

In [ ]:
final_df.head(3)

In [ ]:
final_df.to_csv("juiceWRLDmusic.csv")

### Halsey

In [ ]:
name = "Halsey"
result = sp.search(name) 

In [ ]:
result['tracks']['items'][0]['artists']

In [ ]:
#Extract Artist's uri
artists_uris = result['tracks']['items'][0]['artists'][0]['uri']

In [ ]:
# this is of chainsmokers
artists_uris

In [ ]:
#Extract Artist's uri
artists_uris = result['tracks']['items'][1]['artists'][0]['uri']

In [ ]:
artists_uris

In [ ]:
#Pull all of the artist's albums
artist_albums = sp.artist_albums(artists_uris, album_type='album')

In [ ]:
#Store artist's albums' names' and uris in separate lists
artist_album_names = []
artist_album_uris = []
for i in range(len(artist_albums['items'])):
    artist_album_names.append(artist_albums['items'][i]['name'])
    artist_album_uris.append(artist_albums['items'][i]['uri'])
    
artist_album_names
#Keep names and uris in same order to keep track of duplicate albums

In [ ]:
def album_songs(uri):
    album = uri 
    spotify_albums[album] = {}
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] 
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    #pull data on album tracks
    tracks = sp.album_tracks(album) 
    for n in range(len(tracks['items'])): 
        spotify_albums[album]['album'].append(artist_album_names[album_count]) 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [ ]:
spotify_albums = {}
album_count = 0
for i in artist_album_uris: #each album
    album_songs(i)
    print(str(artist_album_names[album_count]) + " album songs has been added to spotify_albums dictionary")
    album_count+=1 #Updates album count once all tracks have been added

In [ ]:
import time
import numpy as np
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    audio_features(i)
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " playlists completed")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

In [ ]:
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])
        
len(dic_df['album'])

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(dic_df)
df[0:5]

In [ ]:
print(len(df))
final_df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
print(len(final_df))

In [ ]:
final_df.head(3)

In [ ]:
final_df.to_csv("halseyMusic.csv")

### The Weeknd

In [ ]:
name = "The Weeknd"
result = sp.search(name) 
result['tracks']['items'][1]['artists']

In [ ]:
result['tracks']['items'][0]['artists']

In [ ]:
#Extract Artist's uri
artists_uris = result['tracks']['items'][0]['artists'][1]['uri']
artists_uris

In [ ]:
#Pull all of the artist's albums
artist_albums = sp.artist_albums(artists_uris, album_type='album')

#Store artist's albums' names' and uris in separate lists
artist_album_names = []
artist_album_uris = []
for i in range(len(artist_albums['items'])):
    artist_album_names.append(artist_albums['items'][i]['name'])
    artist_album_uris.append(artist_albums['items'][i]['uri'])
    
artist_album_names
#Keep names and uris in same order to keep track of duplicate albums

In [ ]:
def album_songs(uri):
    album = uri 
    spotify_albums[album] = {}
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] 
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    #pull data on album tracks
    tracks = sp.album_tracks(album) 
    for n in range(len(tracks['items'])): 
        spotify_albums[album]['album'].append(artist_album_names[album_count]) 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [ ]:
spotify_albums = {}
album_count = 0
for i in artist_album_uris: #each album
    album_songs(i)
    print(str(artist_album_names[album_count]) + " album songs has been added to spotify_albums dictionary")
    album_count+=1 #Updates album count once all tracks have been added

In [ ]:
def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        
        #Append to relevant key-value
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [ ]:
import time
import numpy as np
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    audio_features(i)
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " playlists completed")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

In [ ]:
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])
        
len(dic_df['album'])

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(dic_df)
df[0:5]

In [ ]:
print(len(df))
final_df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
print(len(final_df))

In [ ]:
final_df.to_csv("theWeekndMusic.csv")

### Post Malone | Doja Cat | Britney Spears | Jon Bellion | Martin Garrix | Maroon 5 | The Chainsmokers | Drake | Ariana Grande | Selena Gomex | Madonna | Nicki Minaj | Cardi B | Bebe Rexha | Megan Thee Stallion | 21 Savage | The Vamps | Kygo | Kesha | Camilla Cabello | Dua Lipa | Charlie Puth | Ed Sheeran | Shawn Mendes | Demi Lovato | Travis Scott | Troye Sivan | Taylor Swift | Khalid | Kanye West | Justin Bieber | French Montana | Lil Tecca | G Eazy | Lil Nas X | Billie Eilish | Olivia Rodrigo | The Kid Laroi | Benny Blanco | Bazzi | Marshmello | Calvin Harris | DVLM | Avicci | Blackbear | Imagine Dragons | Internet Money | Tove Lo | 5SOS | Ellie Goulding | Duncan Laurence 


### Jeremy Zucker | Conan Gray | Kendrick Lamar | Tyga | Lana Del Rey | NF | One Republic | Coldplay | Bryan Adams | Childish Gambino | Metro Boomin | 6Black | A Boogie With Da Hoodie | Logic | KSI | Kodak Black | Chelsea Cutler | Bruno Mars | FUTURE | ----Tate McRae---- | Beyonce | Adele | Lady Gaga | Katy Perry | Alicia Keys | Rihanna | Mariah Carey | ---P!nk--- | Shakira | Miley Cyrus | Avril Lavigne | Tina Turner | Kelly Clarkson | Sia | Jennifer Lopez | Alesso | Alessia Cara

### Lauv | Alec Benjamin | MNEK | Jax Jones | AC/DC | Beatles | Def Leppard | Dolly Parton | Charlie XCX | Galantis | Tai Verdes | Ruel | DNCE | Surfaces | Galantis | Tai Verdes | Ruel | DNCE | Surfaces | Pitbull | Enrique Igleasis | Ruth B . | David Guetta | Little Mix | Kygo |  AVB | Skrillex | Alan Walker | Akon | DJ Snake | Flo Rida | Hardwell | Afrojack | Nicki Romero | R3HAB | Zedd | Headhunterz | Alesso | --Ummet Ozcon-- | Oliver Heldens | Don Diablo | Tiesto | Becky Hill | DJ Khaled | J Balvin | T Pain | --W&W-- | Harry Styles | Gryffin | --Dylan Conrique-- | Travis Barker | Miley Cyrus | --Glass Animals-- | Maren Morris | Olivia O'Brien | Bryson Tiller | Lucky Daye | James Bay | --Normani-- | Anitta | Clean Bandit | Bad Bunny | Zayn Malik | James Young | Becky G | BTS | --lisa-- | --Why dont we-- | Swedish House Mafia | dhruv | Sam smith | Lukas Graham | Sabrina Carpenter | The Script | Anne Marie | Masked Wolf | Usher | Twenty One Pilots | Knife Party | Zomboy | --Kill the Noise-- | Swae Lee | Ludacris | 

In [541]:
name = "Ludacris"
result = sp.search(name) 
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'},
  'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s',
  'id': '1uNFoZAHBGtllmzznpCI3s',
  'name': 'Justin Bieber',
  'type': 'artist',
  'uri': 'spotify:artist:1uNFoZAHBGtllmzznpCI3s'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3ipn9JLAPI5GUEo4y4jcoi'},
  'href': 'https://api.spotify.com/v1/artists/3ipn9JLAPI5GUEo4y4jcoi',
  'id': '3ipn9JLAPI5GUEo4y4jcoi',
  'name': 'Ludacris',
  'type': 'artist',
  'uri': 'spotify:artist:3ipn9JLAPI5GUEo4y4jcoi'}]

In [542]:
#Extract Artist's uri
artists_uris = result['tracks']['items'][0]['artists'][1]['uri']

#Pull all of the artist's albums
artist_albums = sp.artist_albums(artists_uris, album_type='album')

#Store artist's albums' names' and uris in separate lists
artist_album_names = []
artist_album_uris = []
for i in range(len(artist_albums['items'])):
    artist_album_names.append(artist_albums['items'][i]['name'])
    artist_album_uris.append(artist_albums['items'][i]['uri'])
    
artist_album_names
#Keep names and uris in same order to keep track of duplicate albums

['Ludaversal (Deluxe)',
 'Ludaversal (Deluxe)',
 "The Conjure Mixtape: A Hustler's Spirit",
 'Battle Of The Sexes (Deluxe)',
 'Battle Of The Sexes (Deluxe)',
 'Theater Of The Mind (Expanded Edition)',
 'Theater Of The Mind (Expanded Edition)',
 'Release Therapy',
 'Release Therapy',
 'Release Therapy',
 'Ludacris Presents...Disturbing Tha Peace',
 'Ludacris Presents...Disturbing Tha Peace',
 'The Red Light District',
 'The Red Light District',
 'The Red Light District',
 'Chicken - N - Beer',
 'Chicken - N - Beer',
 'Chicken - N - Beer',
 'Word Of Mouf',
 'Word Of Mouf']

In [543]:
def album_songs(uri):
    album = uri 
    spotify_albums[album] = {}
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] 
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    #pull data on album tracks
    tracks = sp.album_tracks(album) 
    for n in range(len(tracks['items'])): 
        spotify_albums[album]['album'].append(artist_album_names[album_count]) 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

spotify_albums = {}
album_count = 0
for i in artist_album_uris: #each album
    album_songs(i)
    print(str(artist_album_names[album_count]) + " album songs has been added to spotify_albums dictionary")
    album_count+=1 #Updates album count once all tracks have been added

Ludaversal (Deluxe) album songs has been added to spotify_albums dictionary
Ludaversal (Deluxe) album songs has been added to spotify_albums dictionary
The Conjure Mixtape: A Hustler's Spirit album songs has been added to spotify_albums dictionary
Battle Of The Sexes (Deluxe) album songs has been added to spotify_albums dictionary
Battle Of The Sexes (Deluxe) album songs has been added to spotify_albums dictionary
Theater Of The Mind (Expanded Edition) album songs has been added to spotify_albums dictionary
Theater Of The Mind (Expanded Edition) album songs has been added to spotify_albums dictionary
Release Therapy album songs has been added to spotify_albums dictionary
Release Therapy album songs has been added to spotify_albums dictionary
Release Therapy album songs has been added to spotify_albums dictionary
Ludacris Presents...Disturbing Tha Peace album songs has been added to spotify_albums dictionary
Ludacris Presents...Disturbing Tha Peace album songs has been added to spotify_

In [544]:
def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        
        #Append to relevant key-value
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

import time
import numpy as np
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    audio_features(i)
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " playlists completed")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

5 playlists completed
Loop #: 5
Elapsed Time: 27.695420503616333 seconds
10 playlists completed
Loop #: 10
Elapsed Time: 52.71953058242798 seconds
15 playlists completed
Loop #: 15
Elapsed Time: 79.66056203842163 seconds
20 playlists completed
Loop #: 20
Elapsed Time: 108.07808446884155 seconds


In [545]:
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])
        
len(dic_df['album'])

329

In [546]:
import pandas as pd
df = pd.DataFrame.from_dict(dic_df)
df[0:5]

,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,Ludaversal (Deluxe),1,77zeaNi0MfXEga1W3Sozbt,Ludaversal Intro,spotify:track:77zeaNi0MfXEga1W3Sozbt,0.05960,0.768,0.823,0.0,0.339,-6.333,0.0978,135.967,0.406,37
1,Ludaversal (Deluxe),2,2cyrYMg4rxY8WNct3lVBm8,Grass Is Always Greener,spotify:track:2cyrYMg4rxY8WNct3lVBm8,0.08470,0.662,0.805,0.0,0.387,-7.043,0.3260,94.929,0.614,40
2,Ludaversal (Deluxe),3,1SzUjM4OtrcNMeDnL0aYDw,Call Ya Bluff,spotify:track:1SzUjM4OtrcNMeDnL0aYDw,0.02250,0.847,0.616,0.0,0.449,-8.138,0.2510,121.992,0.718,45
3,Ludaversal (Deluxe),4,6QkRTkarDVgHf2WGGcmU5g,Lyrical Healing,spotify:track:6QkRTkarDVgHf2WGGcmU5g,0.12500,0.516,0.654,0.0,0.196,-9.746,0.3640,83.193,0.715,31
4,Ludaversal (Deluxe),5,1bMk013HdX3XLG2RNdkswq,Beast Mode,spotify:track:1bMk013HdX3XLG2RNdkswq,0.00104,0.487,0.808,0.0,0.395,-6.559,0.3210,150.555,0.512,39


In [547]:
print(len(df))
final_df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
print(len(final_df))

329
216


In [548]:
final_df.to_csv("ludacris.csv")